In [126]:
import json
import re
import pandas as pd
from pandas.io.json import json_normalize

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 110)

In [127]:
def flattenJSON(json_obj):   
    data = json.loads(json_obj)
    
    def flatten_dict(data):
        for k in list(data):
            if isinstance(data[k], dict):
                child = flatten_dict(data[k])
                for k_ in child:
                    data[k + '.' + k_] = re.sub('^u?\'(.*?)\'','\g<1>',child[k_])
                else:
                    del data[k]
        return data
    
    return flatten_dict(data)

In [128]:
def toConvertableJson(s):    
    def _convertableJson(matchObj):
        if matchObj.group(1):
            return '"'
        elif matchObj.group(2):
            return ':"False"'
        elif matchObj.group(3):
            return ':"True"'
    
    d = flattenJSON(s)
    for i,v in enumerate(d):
        if isinstance(d[v], str) and ('{' in d[v]):
            d[v] = json.loads(re.sub('(\')|(: False)|(: True)',_convertableJson, d[v]))
    return flattenJSON(json.dumps(d))

In [7]:
user[user['useful']>1000]

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
4,MM4RJAeH6yuaN8oZDSt0RA,Nancy,361,2013-10-23 07:02:50,1114,279,665,"2015,2016,2017,2018","mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",39,4.08,28,1,0,0,1,16,57,80,80,25,5
5,0rK89TS8xqy1wI4nYI1wfw,Marilyn,214,2011-06-23 08:05:13,3475,2424,3048,"2015,2016,2017,2018","AJxDPGVTzefy3vSHWJchvQ, TgveR4qIsr0Q7b4MJK29TA...",186,4.20,350,46,50,44,3,221,404,483,483,149,195
6,TEtzbpgA2BFBrC0y0sCbfw,Keane,1122,2006-02-15 18:29:35,13311,19356,15319,"2006,2007,2008,2009,2010,2011,2012,2013","RJQTcJVlBsJ3_Yo0JSFQQg, GWt_h78k1CBBkE1NpThGfQ...",696,4.39,5233,314,333,889,152,1600,3635,4991,4991,1086,428
8,T0gWkTHWRChVUe_Dn1F8nw,Tanya,859,2005-07-20 22:38:17,1630,693,1244,"2006,2007,2008,2009,2010,2011,2012","ctr_BlCf3Ogny-vLs8E9tQ, c6HT44PKCaXqzN_BdgKPCw...",57,4.21,60,16,7,7,3,31,72,95,95,34,2
16,AWCY8laHjH0-3HMT0LGpUA,Jin,732,2010-07-09 15:53:01,1104,417,586,"2011,2012,2013,2014,2015,2016","Xqo1ru1F7srvbUJaCNHkvQ, v5LvxWrscDM7XKpD6f0WxQ...",71,3.79,89,8,11,18,8,83,196,185,185,56,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515504,YZ_i4jzErjP7cgD0LNQJTQ,Maria,82,2014-11-07 05:20:36,2024,1231,2064,"2017,2018","aVU4oPsnZIzIFkqrwRYggw, oazuTtSyKV6zrhbsgbdbUQ...",91,4.01,528,6,18,10,0,121,435,693,693,38,679
1527647,tsMF0FcFcHZ8i28WzWtQXw,Susan,212,2013-05-31 15:15:04,7715,4034,6597,"2017,2018","UjQHMwfao-QywgOVpw0mVQ, TH6JMI7Sd0M3h7XwfTCkgg...",186,3.86,2676,53,88,15,5,1342,3639,3433,3433,301,754
1530072,DwvFoYerl1z0HcwOU2lapw,JP,17,2010-08-08 16:39:49,1267,2,1253,,"7ot0zYlKobDKwIXI7hs3HA, PGhVUifHswX5TPMBOq8rDw...",0,4.82,0,0,0,0,0,2,0,0,0,0,0
1555217,duAatEAMZzqUkPrVtJqU5A,Kathy,53,2014-07-25 01:18:39,1126,579,1061,2018,"amrG2DLedERcpbIcHwKPtw, 1NJMJ0FAg6x0cdgZOF_H4Q...",41,4.30,407,5,10,4,0,166,309,493,493,57,398


### Business

In [131]:
with open('yelp_dataset/business.json') as f:
    data = [toConvertableJson(line) for line in f]
business = pd.DataFrame(data)
#business = business.sample(frac=0.05, replace=False, random_state=1)
business.drop(['hours','attributes'], axis=1, inplace=True) # null

In [136]:
#business[business['categories'].apply(lambda x:'Coffee & Tea' in x if x is not None else False)]

In [132]:
restaurant = business[business['categories'].apply(lambda x:('Restaurants' in x) or (('Bars' in x) and ('Cigar Bars' not in x) and ('Oxygen Bars' not in x)) or ('Food' in x) or ('Coffee & Tea' in x) or ('Fast Food' in x) or ('Burger' in x) or ('Pizza' in x) or ('Sandwiches' in x) or ('Breakfast & Brunch' in x) if x is not None else False)]
restaurant = restaurant[restaurant['city']=='Las Vegas']

#### drop features(core)

In [137]:
for i in restaurant.columns:
    if 'Hair' in i:
        restaurant.drop(restaurant[(restaurant[i]=='False') | (restaurant[i]=='True')]['business_id'].index, axis=0, inplace=True)
        restaurant.drop(columns=i, axis=1, inplace=True)

/Users/necronia/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### drop features(for training)

In [139]:
for i in restaurant.columns:
    if ('hour' in i) or ('Wheelchair' in i):
        restaurant.drop(columns=i, axis=1, inplace=True)

In [181]:
restaurant[restaurant['categories'].apply(lambda x:('Shoe Stores' in x) )]['categories'].values

array(['Jewelry, Shopping, Flowers & Gifts, Shoe Stores, Fashion, Gift Shops, Food, Souvenir Shops, Toy Stores, Shopping Centers, Beer, Wine & Spirits'],
      dtype=object)

#### check category

(8869, 90)

In [183]:
cat = []
[cat.extend(n.split(',')) for n in restaurant['categories'] if n is not None ]
cat = [n.strip() for n in cat]
cat = [(n, cat.count(n)) for n in set(cat)]
cat.sort(key=lambda e:e[1], reverse=True)
cat

[('Restaurants', 6450),
 ('Food', 3594),
 ('Nightlife', 1623),
 ('Bars', 1542),
 ('Fast Food', 985),
 ('American (Traditional)', 896),
 ('Mexican', 827),
 ('Coffee & Tea', 766),
 ('Sandwiches', 741),
 ('Pizza', 686),
 ('American (New)', 677),
 ('Burgers', 622),
 ('Breakfast & Brunch', 543),
 ('Specialty Food', 541),
 ('Desserts', 481),
 ('Chinese', 475),
 ('Italian', 454),
 ('Event Planning & Services', 444),
 ('Shopping', 434),
 ('Seafood', 398),
 ('Japanese', 392),
 ('Arts & Entertainment', 386),
 ('Lounges', 372),
 ('Grocery', 367),
 ('Convenience Stores', 360),
 ('Bakeries', 351),
 ('Ice Cream & Frozen Yogurt', 337),
 ('Asian Fusion', 328),
 ('Wine & Spirits', 309),
 ('Juice Bars & Smoothies', 309),
 ('Beer', 309),
 ('Sports Bars', 298),
 ('Sushi Bars', 283),
 ('Chicken Wings', 269),
 ('Cafes', 265),
 ('Salad', 259),
 ('Steakhouses', 254),
 ('Barbeque', 246),
 ('Caterers', 216),
 ('Gas Stations', 210),
 ('Cocktail Bars', 209),
 ('Delis', 206),
 ('Automotive', 189),
 ('Pubs', 189),


#### check null

In [ ]:
s = pd.DataFrame({'unique_count':restaurant.nunique(),'null_count':restaurant.isnull().sum()})
#s[s['unique_count']==0]
s

In [140]:
restaurant

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories,attributes.GoodForKids,attributes.RestaurantsReservations,attributes.Caters,attributes.NoiseLevel,attributes.RestaurantsTableService,attributes.RestaurantsTakeOut,attributes.RestaurantsPriceRange2,attributes.OutdoorSeating,attributes.BikeParking,attributes.HasTV,attributes.WiFi,attributes.Alcohol,attributes.RestaurantsAttire,attributes.RestaurantsGoodForGroups,attributes.RestaurantsDelivery,attributes.GoodForMeal.dessert,attributes.GoodForMeal.latenight,attributes.GoodForMeal.lunch,attributes.GoodForMeal.dinner,attributes.GoodForMeal.brunch,attributes.GoodForMeal.breakfast,attributes.BusinessParking.garage,attributes.BusinessParking.street,attributes.BusinessParking.validated,attributes.BusinessParking.lot,attributes.BusinessParking.valet,attributes.Ambience.romantic,attributes.Ambience.intimate,attributes.Ambience.classy,attributes.Ambience.hipster,attributes.Ambience.divey,attributes.Ambience.touristy,attributes.Ambience.trendy,attributes.Ambience.upscale,attributes.Ambience.casual,attributes.BusinessAcceptsCreditCards,attributes.BusinessAcceptsBitcoin,attributes.ByAppointmentOnly,attributes.AcceptsInsurance,attributes.GoodForDancing,attributes.CoatCheck,attributes.HappyHour,attributes.Music.dj,attributes.Music.background_music,attributes.Music.no_music,attributes.Music.jukebox,attributes.Music.live,attributes.Music.video,attributes.Music.karaoke,attributes.BestNights.monday,attributes.BestNights.tuesday,attributes.BestNights.friday,attributes.BestNights.wednesday,attributes.BestNights.thursday,attributes.BestNights.sunday,attributes.BestNights.saturday,attributes.Ambience,attributes.BusinessParking,attributes.DogsAllowed,attributes.BYOBCorkage,attributes.DriveThru,attributes.Smoking,attributes.AgesAllowed,attributes.Corkage,attributes.BYOB,attributes.GoodForMeal,attributes.Music,attributes.DietaryRestrictions.dairy-free,attributes.DietaryRestrictions.gluten-free,attributes.DietaryRestrictions.vegan,attributes.DietaryRestrictions.kosher,attributes.DietaryRestrictions.halal,attributes.DietaryRestrictions.soy-free,attributes.DietaryRestrictions.vegetarian,attributes.BestNights,attributes.Open24Hours,attributes.DietaryRestrictions,attributes.RestaurantsCounterService
17,PZ-LZzSlhSe9utkQYU8pFg,Carluccio's Tivoli Gardens,"1775 E Tropicana Ave, Ste 29",Las Vegas,NV,89119,36.100016,-115.128529,4.0,40,0,"Restaurants, Italian",True,True,True,quiet,NaN,True,2,False,NaN,False,no,full_bar,casual,True,False,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,True,False,True,False,True,False,False,False,False,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,KWywu2tTEPWmR9JnBc0WyQ,Hunk Mansion,6007 Dean Martin Dr,Las Vegas,NV,89118,36.080168,-115.182756,4.0,107,1,"Nightlife, Arts & Entertainment, Bars, Strip C...",False,True,NaN,loud,NaN,NaN,2,True,False,True,NaN,full_bar,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,True,True,False,True,False,False,False,False,False,False,False,True,NaN,NaN,NaN,True,NaN,NaN,True,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,tstimHoMcYbkSC4eBA1wEg,Maria's Mexican Restaurant & Bakery,6055 E Lake Mead Blvd,Las Vegas,NV,89156,36.195615,-115.040529,4.5,184,1,"Mexican, Restaurants, Patisserie/Cake Shop, Fo...",True,False,False,average,NaN,True,1,False,True,True,no,beer_and_wine,casual,True,False,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43,NBn4hgfGtNz91k3VsDZlmw,Three Square,4190 N Pecos Rd,Las Vegas,NV,89115,36.236053,-1

In [141]:
restaurant.to_csv('./yelp_dataset/restaurant.csv')

In [66]:
restaurant = pd.read_csv('yelp_dataset/restaurant.csv')

/Users/necronia/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (84,85,86,87,88,89,90,91,103,105,106,107) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [91]:
#[n for n in restaurant.columns if 'Hair' in n]

In [103]:
for i in restaurant.columns:
    print(restaurant[i].value_counts())

ZFT0rf0d4yYiC4fbJ60HNg    1
N9qLVYXE0HzYRE3C0TnzlQ    1
A4MdFCkxy2Es8aGG5ANT3w    1
PJ1vi56FGCvQMyfYvnAqgA    1
tpCALADHTuBaCJNgAluj2w    1
                         ..
NYagmOZuL6Z_4Z1dafyXeQ    1
w3qgHBMr-71i8icNonTEyQ    1
zxzRq9EjkN822nqykqidBw    1
Tsu-FzDz2_xiGVZA3OcvHQ    1
ybKhwymik8zofQTJnhGifw    1
Name: business_id, Length: 77289, dtype: int64
Starbucks                  1066
McDonald's                  806
Subway                      768
Tim Hortons                 333
Pizza Hut                   320
                           ... 
Snowbowl Ice Cream            1
Down the Road Lounge          1
Royal Lounge Restaurant       1
Maui Rose Hawaiian BBQ        1
Dilworth Tasting Room         1
Name: name, Length: 51610, dtype: int64
                                     851
5757 Wayne Newton Blvd                56
3799 Las Vegas Blvd S                 38
5501 Josh Birmingham Pkwy             36
100 City Centre Drive                 35
                                    ... 
331 Ade

no             762
yes_free       499
yes_corkage    158
None             3
Name: attributes.BYOBCorkage, dtype: int64
False    2245
True      944
None        3
Name: attributes.DriveThru, dtype: int64
no         1633
outdoor    1162
yes         300
None          9
Name: attributes.Smoking, dtype: int64
21plus     50
18plus     12
19plus     11
allages     9
None        2
Name: attributes.AgesAllowed, dtype: int64
False    524
True     138
Name: attributes.Corkage, dtype: int64
False    27
True      1
Name: attributes.BYOB, dtype: int64
None    211
Name: attributes.GoodForMeal, dtype: int64
None    65
Name: attributes.Music, dtype: int64
False    50
True      2
Name: attributes.DietaryRestrictions.dairy-free, dtype: int64
False    28
True     24
Name: attributes.DietaryRestrictions.gluten-free, dtype: int64
True     27
False    25
Name: attributes.DietaryRestrictions.vegan, dtype: int64
False    52
Name: attributes.DietaryRestrictions.kosher, dtype: int64
False    49
True      3
Name: 

### Review

In [ ]:
with open('yelp_dataset/review.json') as f:
    data = [json.loads(line) for line in f]
review = pd.DataFrame(data)
#review = review.sample(frac=0.05, replace=False, random_state=1)

In [169]:
review

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38
...,...,...,...,...,...,...,...,...,...
6685895,0pCaq1hqAViS2N6ldlp_sQ,Xu_YazIajfsc7SRj1wJ6Mg,RXBFk3tVBxiTf3uOt9KExQ,5.0,1,0,0,I have been coming here for years and this pla...,2018-07-03 12:17:27
6685896,jPCXuSBbI_cPocgoNBhdkg,V6BjjQICDO4q7TT3ZhaPWw,yA6dKNm_zl1ucZCnwW8ZCg,1.0,13,11,5,I think this owner and the owner of Amy's Baki...,2013-05-22 15:32:08
6685897,zwzO3yPdGbdgGy3-XQ0SWg,PFiIECX8wuvi7P-1mCvIjA,a192hdM0_UVCYLwPJv1Qwg,5.0,0,0,0,"Off the grid Mexican in Vegas. Very tasty, qua...",2018-11-13 21:15:20
6685898,o8Nc2BJhKJXM6tRHJDRuPA,wu5R5_N7q5iqCBh6NBX26Q,kOo4ZY2UQAX4j312mzQ8mA,5.0,1,1,1,We hired Taco Naco to cater our family party a...,2018-08-13 18:09:16


In [142]:
#review_restaurant = review[review['business_id'].apply(lambda x:x in list(restaurant['business_id']))]
res_k = restaurant[['business_id']]
review_restaurant = res_k.merge(review, how='left',on='business_id')

In [143]:
review_restaurant.to_csv('./yelp_dataset/review_restaurant.csv')

In [144]:
review_restaurant

,business_id,review_id,user_id,stars,useful,funny,cool,text,date
0,PZ-LZzSlhSe9utkQYU8pFg,klcF45wKIOpJW_BhJslOJg,-Yz2wIcsdJxUOFMbTgoKQA,5.0,1,0,0,"We went there for dinner the other night, bein...",2011-06-29 02:55:07
1,PZ-LZzSlhSe9utkQYU8pFg,Li-pQG6A7p5gbgZHTMeDSQ,jYcf_e5p0UG0S-9gJq_tNA,4.0,1,0,0,i had the best Chicken Marcela ever. The spagh...,2010-10-06 18:20:13
2,PZ-LZzSlhSe9utkQYU8pFg,iRLX3dJ3ONvncIxPnXy1cw,nQC0JiPIk_jCooRDxpuw5A,5.0,1,0,0,Basically the best Italian in town for the pri...,2010-01-13 00:35:45
3,PZ-LZzSlhSe9utkQYU8pFg,rklteWf9xnTU3fAtMFBRRw,Gv_-mtOKhWFtCjn9xFe0SQ,3.0,1,0,0,Mmmmm delicious food and a little history. Mr....,2008-08-23 20:30:33
4,PZ-LZzSlhSe9utkQYU8pFg,UfRqM0RGdZa86hFcFEAnjw,pabMYegF28KjHQ5hybAJ0A,3.0,1,0,0,"This is old Vegas, this atmosphere is old scho...",2009-06-01 20:02:55
...,...,...,...,...,...,...,...,...,...
1386319,vIAEWbTJc657yN8I4z7whQ,-SEmE5vSGN8VDUF18JRczA,8QKrhvVqhEkD8xo4E4s0GQ,5.0,1,0,2,The Baristas rock here! I stop in on my way to...,2012-10-05 22:04:07
1386320,vIAEWbTJc657yN8I4z7whQ,-sx8tpdUukgvh0uByg4DBQ,9ks-80ZxeCZWNObzEbW4-Q,1.0,3,4,1,This is THE worst Starbucks I've ever been to....,2015-04-01 04:34:41
1386321,vIAEWbTJc657yN8I4z7whQ,82qiOWCRRPTjRcxcIq018g,EA-UgJmd-hQ8RBn1odmDJg,3.0,0,0,0,The guy here at window just turned this whole ...,2015-03-31 21:08:21
1386322,vIAEWbTJc657yN8I4z7whQ,4U49OBNRHvjm62hcBEKy2A,bcxc4OF5SDiJ4xnNMBZUQQ,5.0,0,0,0,I honestly don't know why this location has su...,2017-12-28 06:26:37


### User

In [ ]:
with open('yelp_dataset/user.json') as f:
    data = [json.loads(line) for line in f]
user = pd.DataFrame(data)
#user = user.sample(frac=0.05, replace=False, random_state=1)

In [146]:
user

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,l6BmjZMeQD3rDxWUbiAiow,Rashmi,95,2013-10-08 23:11:33,84,17,25,"2015,2016,2017","c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",5,4.03,2,0,0,0,0,1,1,1,1,2,0
1,4XChL029mKr5hydo79Ljxg,Jenna,33,2013-02-21 22:29:06,48,22,16,,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",4,3.63,1,0,0,0,0,0,0,1,1,0,0
2,bc8C_eETBWL0olvFSJJd0w,David,16,2013-10-04 00:16:10,28,8,10,,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",0,3.71,0,0,0,0,0,1,0,0,0,0,0
3,dD0gZpBctWGdWo9WlGuhlA,Angela,17,2014-05-22 15:57:30,30,4,14,,"RZ6wS38wnlXyj-OOdTzBxA, l5jxZh1KsgI8rMunm-GN6A...",5,4.85,1,0,0,0,0,0,2,0,0,1,0
4,MM4RJAeH6yuaN8oZDSt0RA,Nancy,361,2013-10-23 07:02:50,1114,279,665,"2015,2016,2017,2018","mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",39,4.08,28,1,0,0,1,16,57,80,80,25,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637133,oC4BjbB4I-wx5KBAO1k2Gg,Jiaxin,2,2016-02-24 19:28:47,0,0,0,,None,0,4.50,0,0,0,0,0,0,0,0,0,0,0
1637134,LKd_-BT2GMWp8_1ho18kOg,Wes,8,2017-02-25 23:34:05,0,0,0,,None,0,4.00,0,0,0,0,0,0,0,0,0,0,0
1637135,ib_elCBhGOamS0AALkh4Og,Lee,4,2009-08-19 22:08:16,8,0,0,,None,0,3.40,0,0,0,0,0,0,0,0,0,0,0
1637136,WV27LoSFDG_YmiDKTLKKLA,Melissa,1,2018-10-15 17:07:43,1,0,0,,None,0,3.00,0,0,0,0,0,0,0,0,0,0,0


In [151]:
rev_k = pd.DataFrame({'user_id':review_restaurant['user_id'].unique()})
user_restaurant = rev_k.merge(user, how='left',on='user_id')

In [152]:
user_restaurant

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,-Yz2wIcsdJxUOFMbTgoKQA,Walt,72,2011-06-29 02:38:05,177,35,28,,"Sh6E0df5Y1Em-MHDeL6Nkw, 8SgjU2fID9VqKQooFt1DIQ...",2,3.42,1,0,0,0,0,2,1,3,3,0,0
1,jYcf_e5p0UG0S-9gJq_tNA,Fred,21,2009-02-05 00:22:04,32,9,5,,None,1,2.86,0,0,0,0,0,0,2,0,0,0,0
2,nQC0JiPIk_jCooRDxpuw5A,N,18,2010-01-12 23:10:51,34,16,2,,None,0,3.89,0,0,0,0,0,1,0,0,0,0,0
3,Gv_-mtOKhWFtCjn9xFe0SQ,Crysta,149,2008-06-23 19:08:04,170,59,81,,"pWVy4iCapVG-3DDb6celgA, GGWjA8v479feCo6OPTLQOg...",24,4.05,26,5,1,3,1,6,7,10,10,2,0
4,pabMYegF28KjHQ5hybAJ0A,Steve,59,2009-03-03 16:53:04,45,10,12,,"KGBfVdXlMRP9OoP8PusISg, mJLRvqLOKhqEdkgt9iEaCQ...",2,3.80,1,0,0,0,0,0,0,3,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464390,_77s6LTBJ4BrDoR1phYiWg,Tim,12,2012-02-24 19:31:33,6,0,2,,None,0,3.47,0,0,0,0,0,0,0,0,0,0,0
464391,cSb3VUePA0vk_oH8-_nAMg,Aaron,5,2012-03-05 21:04:21,0,0,1,,"ipKpAtbJiziTg2uCnf2mDQ, tgfTT6wbnOmsuMs52TbeEA...",0,4.20,0,0,0,0,0,0,0,0,0,0,0
464392,5F_1TgQWAncO2qkodeu2PA,Ashley,1,2014-11-24 21:49:22,0,0,0,,None,0,1.00,0,0,0,0,0,0,0,0,0,0,0
464393,bmANZOzkp7ztoFhdPdnjHg,Rose,15,2011-01-27 00:30:21,20,11,9,,None,0,3.47,1,0,0,0,0,0,1,0,0,0,0


#### NLP

In [109]:
import nltk
from gensim.models.word2vec import Word2Vec
import string

In [110]:
#stopword 목록 다운로드
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/necronia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [112]:
translator = str.maketrans('', '', string.punctuation)

In [118]:
review_restaurant['text'] = review_restaurant['text'].astype(str)

In [119]:
# each.translate(translator) == 특수문자 제거
# x.lower() == 소문자화
# if x.lower() not in stop_words == 불용어제거
clean = [[x.lower() for x in each.translate(translator).split() if x.lower() not in stop_words] for each in review_restaurant['text'].values]

In [123]:
clean

[['girlfriend',
  'went',
  'dinner',
  'emerald',
  'chinese',
  'thursday',
  'night',
  'workout',
  'arrived',
  'around',
  '730pm',
  'seated',
  'short',
  'wait',
  'browsing',
  'menu',
  'brief',
  'discussion',
  'waiter',
  'ordered',
  'seafood',
  'noodles',
  '3',
  'item',
  'mix',
  'meat',
  'plate',
  'bbq',
  'pork',
  'chicken',
  'duck',
  'food',
  'came',
  'soon',
  'thereafter',
  'hot',
  'delicious',
  'large',
  'portions',
  'lots',
  'seafood',
  'seafood',
  'noodles',
  'mix',
  'meat',
  'plate',
  'came',
  'good',
  'cuts',
  'meat',
  'correct',
  'sauces',
  'meal',
  'received',
  'fortune',
  'cookies',
  'dessert',
  'food',
  'food',
  'good',
  'quality',
  'ingredients',
  'tasted',
  'fresh',
  'dishes',
  'tasted',
  'well',
  'made',
  'food',
  'average',
  'nothing',
  'stood',
  'even',
  'though',
  'quite',
  'hungry',
  'price',
  'price',
  'high',
  'comparative',
  'chinese',
  'cuisine',
  'comparable',
  'large',
  'western',
  

In [171]:
#### it takes too long.

# model = Word2Vec(clean,window=5,min_count=100,sg=1,iter=1000)

# print(list(model.wv.vocab.keys()))
# print("vocab length : %d"%len(model.wv.vocab))

In [170]:
review_restaurant['text'].map(lambda x : len(x.split(' '))).describe()

count    1.386324e+06
mean     1.123440e+02
std      1.075050e+02
min      1.000000e+00
25%      4.200000e+01
50%      7.800000e+01
75%      1.440000e+02
max      1.702000e+03
Name: text, dtype: float64

In [176]:
review_restaurant

,business_id,review_id,user_id,stars,useful,funny,cool,text,date
0,PZ-LZzSlhSe9utkQYU8pFg,klcF45wKIOpJW_BhJslOJg,-Yz2wIcsdJxUOFMbTgoKQA,5.0,1,0,0,"We went there for dinner the other night, bein...",2011-06-29 02:55:07
1,PZ-LZzSlhSe9utkQYU8pFg,Li-pQG6A7p5gbgZHTMeDSQ,jYcf_e5p0UG0S-9gJq_tNA,4.0,1,0,0,i had the best Chicken Marcela ever. The spagh...,2010-10-06 18:20:13
2,PZ-LZzSlhSe9utkQYU8pFg,iRLX3dJ3ONvncIxPnXy1cw,nQC0JiPIk_jCooRDxpuw5A,5.0,1,0,0,Basically the best Italian in town for the pri...,2010-01-13 00:35:45
3,PZ-LZzSlhSe9utkQYU8pFg,rklteWf9xnTU3fAtMFBRRw,Gv_-mtOKhWFtCjn9xFe0SQ,3.0,1,0,0,Mmmmm delicious food and a little history. Mr....,2008-08-23 20:30:33
4,PZ-LZzSlhSe9utkQYU8pFg,UfRqM0RGdZa86hFcFEAnjw,pabMYegF28KjHQ5hybAJ0A,3.0,1,0,0,"This is old Vegas, this atmosphere is old scho...",2009-06-01 20:02:55
...,...,...,...,...,...,...,...,...,...
1386319,vIAEWbTJc657yN8I4z7whQ,-SEmE5vSGN8VDUF18JRczA,8QKrhvVqhEkD8xo4E4s0GQ,5.0,1,0,2,The Baristas rock here! I stop in on my way to...,2012-10-05 22:04:07
1386320,vIAEWbTJc657yN8I4z7whQ,-sx8tpdUukgvh0uByg4DBQ,9ks-80ZxeCZWNObzEbW4-Q,1.0,3,4,1,This is THE worst Starbucks I've ever been to....,2015-04-01 04:34:41
1386321,vIAEWbTJc657yN8I4z7whQ,82qiOWCRRPTjRcxcIq018g,EA-UgJmd-hQ8RBn1odmDJg,3.0,0,0,0,The guy here at window just turned this whole ...,2015-03-31 21:08:21
1386322,vIAEWbTJc657yN8I4z7whQ,4U49OBNRHvjm62hcBEKy2A,bcxc4OF5SDiJ4xnNMBZUQQ,5.0,0,0,0,I honestly don't know why this location has su...,2017-12-28 06:26:37


In [177]:
review_txt = review_restaurant['text'].to_list()
review_label = review_restaurant['business_id'].to_list()

#### tokenize

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # We will cut reviews after 100 words
training_samples = 20000  # We will be training on 20000 samples
validation_samples = 5000  # We will be validating on 5000 samples
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
# Sequence of all the training samples, with word indexing replacing the actual word.
sequences = tokenizer.texts_to_sequences(texts)
print(sequences[0])

# A sorted dictionary that saves the index of each word, from the most frequent to the least frequent
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)
print(data[0])

### Check in

In [12]:
with open('yelp_dataset/checkin.json') as f:
    data = [json.loads(line) for line in f]
checkin = pd.DataFrame(data)
#checkin = checkin.sample(frac=0.05, replace=False, random_state=1)
checkin

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."
...,...,...
161945,zzvlwkcNR1CCqOPXwuvz2A,"2017-05-06 20:05:15, 2017-05-12 22:37:03, 2017..."
161946,zzwaS0xn1MVEPEf0hNLjew,"2010-02-16 02:09:56, 2010-07-05 05:40:48, 2010..."
161947,zzwhN7x37nyjP0ZM8oiHmw,"2016-03-06 13:27:02, 2016-03-09 00:41:53, 2016..."
161948,zzwicjPC9g246MK2M1ZFBA,"2012-09-22 00:26:15, 2012-09-23 20:12:00, 2012..."


### Tip (not use)

In [13]:
with open('yelp_dataset/tip.json') as f:
    data = [json.loads(line) for line in f]
tip = pd.DataFrame(data)
#tip = tip.sample(frac=0.05, replace=False, random_state=1)
tip

,user_id,business_id,text,date,compliment_count
0,UPw5DWs_b-e2JRBS-t37Ag,VaKXUpmWTTWDKbpJ3aQdMw,"Great for watching games, ufc, and whatever el...",2014-03-27 03:51:24,0
1,Ocha4kZBHb4JK0lOWvE0sg,OPiPeoJiv92rENwbq76orA,Happy Hour 2-4 daily with 1/2 price drinks and...,2013-05-25 06:00:56,0
2,jRyO2V1pA4CdVVqCIOPc1Q,5KheTjYPu1HcQzQFtm4_vw,Good chips and salsa. Loud at times. Good serv...,2011-12-26 01:46:17,0
3,FuTJWFYm4UKqewaosss1KA,TkoyGi8J7YFjA6SbaRzrxg,The setting and decoration here is amazing. Co...,2014-03-23 21:32:49,0
4,LUlKtaM3nXd-E4N4uOk_fQ,AkL6Ous6A1atZejfZXn1Bg,Molly is definately taking a picture with Sant...,2012-10-06 00:19:27,0
...,...,...,...,...,...
1223089,nus5X9JKgU_SzfAL4vhJrg,qdwwiRKwnhdlr9QcgJuzbg,"Very good class, great instructor!!",2018-08-05 20:58:09,0
1223090,95wqYGU6-OiHqRPZP0uAWw,wrs4Zbz17q_G49AIRxRPVw,Excellent food,2018-08-15 18:05:22,0
1223091,HtLaONYabidSxMS1eGHcXA,wEc4JE2NYQr1RnGjGr4q7w,Jess is the best bartender!! Go check her out!,2018-10-02 03:34:58,0
1223092,T4R5ZKfcGt1GIAgjCh67eA,n5Hc_iMVkyxcS0SXiBCbOQ,Excellent food and AMAZING service by Anneliese!!,2018-10-13 16:57:50,0
